In [1]:
import random
import collections
import queue
import argparse

In [2]:
DEFAULT_NUMBER_OF_TAXIS = 3
DEFAULT_END_TIME = 180
SEARCH_DURATION = 5
TRIP_DURATION = 20
DEPARTURE_INTERVAL = 5

In [3]:
Event = collections.namedtuple('Event', 'time proc action')

#### # BEGIN TAXI_PROCESS

In [4]:
def taxi_process(ident, trips, start_time=0):  # <1>
    """Yield to simulator issuing event at each state change"""
    time = yield Event(start_time, ident, 'leave garage')  # <2>
    for i in range(trips):  # <3>
        time = yield Event(time, ident, 'pick up passenger')  # <4>
        time = yield Event(time, ident, 'drop off passenger')  # <5>

    yield Event(time, ident, 'going home')  # <6>
    # end of taxi process # <7>

#### # BEGIN TAXI_SIMULATOR

In [5]:
class Simulator:

    def __init__(self, procs_map):
        self.events = queue.PriorityQueue()
        self.procs = dict(procs_map)

    def run(self, end_time):  # <1>
        """Schedule and display events until time is up"""
        # schedule the first event for each cab
        for _, proc in sorted(self.procs.items()):  # <2>
            first_event = next(proc)  # <3>
            self.events.put(first_event)  # <4>

        # main loop of the simulation
        sim_time = 0  # <5>
        while sim_time < end_time:  # <6>
            if self.events.empty():  # <7>
                print('*** end of events ***')
                break

            current_event = self.events.get()  # <8>
            sim_time, proc_id, previous_action = current_event  # <9>
            print('taxi:', proc_id, proc_id * '   ', current_event)  # <10>
            active_proc = self.procs[proc_id]  # <11>
            next_time = sim_time + compute_duration(previous_action)  # <12>
            try:
                next_event = active_proc.send(next_time)  # <13>
            except StopIteration:
                del self.procs[proc_id]  # <14>
            else:
                self.events.put(next_event)  # <15>
        else:  # <16>
            msg = '*** end of simulation time: {} events pending ***'
            print(msg.format(self.events.qsize()))
# END TAXI_SIMULATOR

In [6]:
def compute_duration(previous_action):
    """Compute action duration using exponential distribution"""
    if previous_action in ['leave garage', 'drop off passenger']:
        # new state is prowling
        interval = SEARCH_DURATION
    elif previous_action == 'pick up passenger':
        # new state is trip
        interval = TRIP_DURATION
    elif previous_action == 'going home':
        interval = 1
    else:
        raise ValueError('Unknown previous_action: %s' % previous_action)
    return int(random.expovariate(1/interval)) + 1

In [7]:
def main(end_time=DEFAULT_END_TIME, num_taxis=DEFAULT_NUMBER_OF_TAXIS,
         seed=None):
    """Initialize random generator, build procs and run simulation"""
    if seed is not None:
        random.seed(seed)  # get reproducible results

    taxis = {i: taxi_process(i, (i+1)*2, i*DEPARTURE_INTERVAL)
             for i in range(num_taxis)}
    sim = Simulator(taxis)
    sim.run(end_time)

In [8]:
if __name__ == '__main__':

    parser = argparse.ArgumentParser(
                        description='Taxi fleet simulator.')
    parser.add_argument('-e', '--end-time', type=int,
                        default=DEFAULT_END_TIME,
                        help='simulation end time; default = %s'
                        % DEFAULT_END_TIME)
    parser.add_argument('-t', '--taxis', type=int,
                        default=DEFAULT_NUMBER_OF_TAXIS,
                        help='number of taxis running; default = %s'
                        % DEFAULT_NUMBER_OF_TAXIS)
    parser.add_argument('-s', '--seed', type=int, default=None,
                        help='random generator seed (for testing)')

    args = parser.parse_args()
    main(args.end_time, args.taxis, args.seed)

usage: ipykernel_launcher.py [-h] [-e END_TIME] [-t TAXIS] [-s SEED]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\Ivo\AppData\Roaming\jupyter\runtime\kernel-c6c6125f-ba25-4951-971d-c68e52f6a6b7.json


SystemExit: 2

C:\Users\Ivo\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3259: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


#### Taxi simulator
#### Driving a taxi from the console::

In [9]:
>>> taxi = taxi_process(ident=13, trips=2, start_time=0)
>>> next(taxi)

Event(time=0, proc=13, action='leave garage')

In [10]:
>>> taxi.send(_.time + 7)

Event(time=7, proc=13, action='pick up passenger')

In [11]:
>>> taxi.send(_.time + 23)

Event(time=30, proc=13, action='drop off passenger')

In [12]:
>>> taxi.send(_.time + 5)

Event(time=35, proc=13, action='pick up passenger')

In [13]:
>>> taxi.send(_.time + 48)

Event(time=83, proc=13, action='drop off passenger')

In [14]:
>>> taxi.send(_.time + 1)

Event(time=84, proc=13, action='going home')

In [15]:
>>> taxi.send(_.time + 10)

StopIteration: 

#### Sample run with two cars, random seed 10. This is a valid doctest::

In [16]:
>>> main(num_taxis=2, seed=10)

taxi: 0  Event(time=0, proc=0, action='leave garage')
taxi: 0  Event(time=5, proc=0, action='pick up passenger')
taxi: 1     Event(time=5, proc=1, action='leave garage')
taxi: 1     Event(time=10, proc=1, action='pick up passenger')
taxi: 1     Event(time=15, proc=1, action='drop off passenger')
taxi: 0  Event(time=17, proc=0, action='drop off passenger')
taxi: 1     Event(time=24, proc=1, action='pick up passenger')
taxi: 0  Event(time=26, proc=0, action='pick up passenger')
taxi: 0  Event(time=30, proc=0, action='drop off passenger')
taxi: 0  Event(time=34, proc=0, action='going home')
taxi: 1     Event(time=46, proc=1, action='drop off passenger')
taxi: 1     Event(time=48, proc=1, action='pick up passenger')
taxi: 1     Event(time=110, proc=1, action='drop off passenger')
taxi: 1     Event(time=139, proc=1, action='pick up passenger')
taxi: 1     Event(time=140, proc=1, action='drop off passenger')
taxi: 1     Event(time=150, proc=1, action='going home')
*** end of events ***


#### See longer sample run at the end of this module.